In [1]:
# pip install scikit-learn,  scipy,  numpy

import numpy as np
import re
from scipy.sparse import csr_matrix, hstack
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.model_selection import KFold, cross_val_score

In [2]:
# Datos de ejemplo

textos = [
    "Me encantó la película!! Fue INCREÍBLE",
    "Horrible servicio, no vuelvo más",
    "Excelente atención, todo perfecto",
    "La comida estaba fría y sin sabor",
    "Muy bueno, me encantó el resultado",
    "PÉSIMO producto, una estafa!!!",
    "Muy malo, no lo recomiendo",
    "Muy malo, no lo recomiendo",
    "Increíble trabajo, felicidades!",
    "Terrible, llegó roto y tarde"
]
etiquetas = [
    "positivo","negativo","positivo","negativo","positivo",
    "negativo","negativo","negativo","positivo","negativo"
]

In [3]:
# TF-IDF, utilizando palabras y pares consevutivos -- ngram_range=(1,2)

vectorizador = TfidfVectorizer(lowercase=True, strip_accents="unicode", ngram_range=(1,2))
X_tfidf = vectorizador.fit_transform(textos)

#palabras - palabras consecutivas
print(vectorizador.get_feature_names_out())

#número de frases,numuero de palabras o palabras consecutivas, lo que se fijo en  ngram_range
print(X_tfidf.shape)
print("TF-IDF")
#Muestra dolo los valores distintos de cero, para mostrar toda la mtriz X_tfidf.toarray()
print(X_tfidf)
#print(X_tfidf.toarray())

['atencion' 'atencion todo' 'bueno' 'bueno me' 'comida' 'comida estaba'
 'el' 'el resultado' 'encanto' 'encanto el' 'encanto la' 'estaba'
 'estaba fria' 'estafa' 'excelente' 'excelente atencion' 'felicidades'
 'fria' 'fria sin' 'fue' 'fue increible' 'horrible' 'horrible servicio'
 'increible' 'increible trabajo' 'la' 'la comida' 'la pelicula' 'llego'
 'llego roto' 'lo' 'lo recomiendo' 'malo' 'malo no' 'mas' 'me'
 'me encanto' 'muy' 'muy bueno' 'muy malo' 'no' 'no lo' 'no vuelvo'
 'pelicula' 'pelicula fue' 'perfecto' 'pesimo' 'pesimo producto'
 'producto' 'producto una' 'recomiendo' 'resultado' 'roto' 'roto tarde'
 'sabor' 'servicio' 'servicio no' 'sin' 'sin sabor' 'tarde' 'terrible'
 'terrible llego' 'todo' 'todo perfecto' 'trabajo' 'trabajo felicidades'
 'una' 'una estafa' 'vuelvo' 'vuelvo mas']
(10, 70)
TF-IDF
  (0, 20)	0.3225256660752875
  (0, 44)	0.3225256660752875
  (0, 27)	0.3225256660752875
  (0, 10)	0.3225256660752875
  (0, 36)	0.274176282101918
  (0, 23)	0.274176282101918
  (0

In [4]:
#Caratarísticas manuales, 
def extraer_caracteristicas(textos):
    data = []
    for t in textos:
        print("Texto: ", t)
        exclamaciones = t.count("!")
        letras = [c for c in t if c.isalpha()] #considera solo letras
        mayusculas = sum(1 for c in letras if c.isupper()) #letras mayusculas
        ratio_mayus = mayusculas / len(letras) if letras else 0 #proporcion letras mayusculas
        data.append([exclamaciones, ratio_mayus])
        print("Exclamaciones: ", exclamaciones)
        print("Mayusculas:    ", mayusculas)
        print("Ratio_May:     ", ratio_mayus, "\n")
    print("Data: ", data)
    #Compressed Sparse Row matrix -- Matruz dispersa/Esparsa comprimida por filas
    return csr_matrix(np.array(data, dtype=float))

X_manual = extraer_caracteristicas(textos)
print("X_Manual\n", X_manual)

Texto:  Me encantó la película!! Fue INCREÍBLE
Exclamaciones:  2
Mayusculas:     11
Ratio_May:      0.3548387096774194 

Texto:  Horrible servicio, no vuelvo más
Exclamaciones:  0
Mayusculas:     1
Ratio_May:      0.037037037037037035 

Texto:  Excelente atención, todo perfecto
Exclamaciones:  0
Mayusculas:     1
Ratio_May:      0.034482758620689655 

Texto:  La comida estaba fría y sin sabor
Exclamaciones:  0
Mayusculas:     1
Ratio_May:      0.037037037037037035 

Texto:  Muy bueno, me encantó el resultado
Exclamaciones:  0
Mayusculas:     1
Ratio_May:      0.03571428571428571 

Texto:  PÉSIMO producto, una estafa!!!
Exclamaciones:  3
Mayusculas:     6
Ratio_May:      0.2608695652173913 

Texto:  Muy malo, no lo recomiendo
Exclamaciones:  0
Mayusculas:     1
Ratio_May:      0.047619047619047616 

Texto:  Muy malo, no lo recomiendo
Exclamaciones:  0
Mayusculas:     1
Ratio_May:      0.047619047619047616 

Texto:  Increíble trabajo, felicidades!
Exclamaciones:  1
Mayusculas:     1
Rati

In [5]:
# Combinaciión de TF-IDF + características manuales
X_final = hstack([X_tfidf, X_manual])
print(X_final.shape)

(10, 72)


In [6]:
#Validación cruzada (K-Fold = 5)

#SVM Lineal
modelo = LinearSVC()
kf = KFold(n_splits=5, shuffle=True, random_state=42)

#F1 macro (equilibra clases positivas/negativas)
scores = cross_val_score(modelo, X_final, etiquetas, cv=kf, scoring='f1_macro')

print("Puntajes F1 por fold:", np.round(scores, 3))
print("F1 promedio:", round(scores.mean(), 3))



Puntajes F1 por fold: [0.333 0.333 0.333 0.333 0.333]
F1 promedio: 0.333


/Users/eliana/anaconda3/lib/python3.11/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/Users/eliana/anaconda3/lib/python3.11/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/Users/eliana/anaconda3/lib/python3.11/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/Users/eliana/anaconda3/lib/python3.11/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/Users/eliana/anacon

In [7]:
# Entrenar el modelo
modelo.fit(X_final, etiquetas)

/Users/eliana/anaconda3/lib/python3.11/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


LinearSVC()

In [8]:
#Predecir. Recordar que estos datos no deben haber sido utilizados en el entrenamiento

nuevos_textos = [
    "EXCELENTE!!!",
    "Malo, una pérdida de dinero, no me gusto",
    "TERRIBLE, muy malo"
]
X_nuevos_tfidf = vectorizador.transform(nuevos_textos)
X_nuevos_manual = extraer_caracteristicas(nuevos_textos)
X_nuevos_final = hstack([X_nuevos_tfidf, X_nuevos_manual])

print("Predicciones:", modelo.predict(X_nuevos_final))


Texto:  EXCELENTE!!!
Exclamaciones:  3
Mayusculas:     9
Ratio_May:      1.0 

Texto:  Malo, una pérdida de dinero, no me gusto
Exclamaciones:  0
Mayusculas:     1
Ratio_May:      0.03225806451612903 

Texto:  TERRIBLE, muy malo
Exclamaciones:  0
Mayusculas:     8
Ratio_May:      0.5333333333333333 

Data:  [[3, 1.0], [0, 0.03225806451612903], [0, 0.5333333333333333]]
Predicciones: ['positivo' 'negativo' 'negativo']
